#### Ali - Assignment

In [1]:
import requests
import sqlite3
from decouple import config
from sqlite3 import Error
from datetime import datetime

def createDB_or_connect(db_file):
    #create a database connection to a SQLite database
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)
    return conn


def create_table(conn, create_table_sql):
    # create a table from the create_table_sql statement
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
        print('table created!')
        c.close()
    except Error as e:
        print(e)

        
def query_db(conn, sql_query):
    # query data from DB
    cur = conn.cursor()
    cur.execute(sql_query)
    
    rows = cur.fetchall()
    
    for row in rows:
        print(row[0])

        
def insert_metric(conn, station_id, end_date, metric, measurement, measurement_type, granularity):
    # insert a new metric
    sql = ''' INSERT INTO weather_metric(station_id,end_date,metric,measurement,measurement_type,granularity)
              VALUES(?,?,?,?,?,?) '''
    try:
        cur = conn.cursor()
        cur.execute(sql, (station_id, end_date, metric, measurement, measurement_type, granularity))
        conn.commit()
    except Error as e:
        print(e)
    return cur.lastrowid


def pull_data(station_id, start_date, end_date, frequency):
    # pull data from API
    assert frequency in ['daily', 'monthly'], frequency
    
    url = 'https://meteostat.p.rapidapi.com/stations/' + frequency

    querystring = {"station":station_id,"start":start_date,"end":end_date, "tz":"Europe/Berlin"}
    
    if frequency == 'monthly':
        querystring["freq"]="M"
    
    api_key = config('api_key',default='')
    headers = {
        'x-rapidapi-key': api_key,
        'x-rapidapi-host': "meteostat.p.rapidapi.com"
        }

    response = requests.request("GET", url, headers=headers, params=querystring)
    return response.json()


def import_historical_monthly_avg_temperature(station_id, start_date, end_date):
    # import historical monthly data from API and store it in DB
    json=pull_data(station_id, start_date, end_date, frequency='monthly')    
    
    if json:
        print('data pulled from the API..')
        
    conn = createDB_or_connect(r"weather.db")
    if conn:
        print('connected to DB..')
    
    for month in json['data']:    
        insert_metric(conn, station_id, month['date'], 'Temperature', month['tavg'], 'avg', 'M')
    conn.close()
    print('Done importing data!')
    
    
def import_daily_avg_temperature(station_id, start_date, end_date):
    # import daily data from API and store it in DB
    json=pull_data(station_id, start_date, end_date, frequency='daily')    
    
    if json:
        print('data pulled from the API..')
        
    conn = createDB_or_connect(r"weather.db")
    if conn:
        print('connected to DB..')
    
    for day in json['data']:    
        insert_metric(conn, station_id, day['date'], 'Temperature', day['tavg'], 'avg', 'D')
    conn.close()
    print('Done importing data!')
    

def import_today_avg_temperature(station_id):
    # import today's data to DB
    today_date = datetime.today().strftime('%Y-%m-%d')
    import_daily_avg_temperature(station_id, today_date, today_date)

In [2]:
# creatre weather database
conn = createDB_or_connect(r"weather.db")

# create weather_metric table with:
# id: incremental id for each record
# station_id: the id of the station at which the metric is recorded
# end_date: the end date at which the measurement is concluded (e.g., for month it's the date of last day)
# metric: the name of the metric (e.g., Temperature)
# measurement: the real-valued number representing the measurement of the metric name
# measurement_type: specifies whether the measurement is a point value or an aggregated value (e.g., avg)
# grandularity: specifies the timespan unit across which the measurement is concluded (e.g., 'M' for month, 'D' for day)

create_temperature_table = """ CREATE TABLE IF NOT EXISTS weather_metric (
                                        id integer PRIMARY KEY AUTOINCREMENT,
                                        station_id integer NOT NULL,
                                        end_date text NOT NULL,
                                        metric text NOT NULL,
                                        measurement real,
                                        measurement_type text,
                                        granularity text NOT NULL
                                    ); """

create_table(conn, create_temperature_table)

table created!


In [3]:
# to identify the station_id for 'Berlin / Tegel', we pinpointed the place on google maps
# and queried the 'nearby' endpoint using at: "https://meteostat.p.rapidapi.com/stations/nearby" 
# passing it in the query string the lat and long of the 'Berlin / Tegel' with limit=1 (to only return that station)

import_historical_monthly_avg_temperature(10382, "1963-01-01", "2021-05-31")

data pulled from the API..
connected to DB..
Done importing data!


In [4]:
sql_query = """
                SELECT AVG(measurement) AS AVG_TEMPERATURE FROM weather_metric 
                WHERE granularity='M' AND SUBSTR(end_date, 6, 2) = '02';
                """
print('Average air temperature for the month of February for the Berlin/Tegel weather station across all available years is:')
query_db(conn, sql_query)

Average air temperature for the month of February for the Berlin/Tegel weather station across all available years is:
1.3661016949152545
